In [1]:
import pandas as pd
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import json
import os
import rasterio
import numpy as np


## cut raster

In [2]:
gdf = gpd.read_file('../../../Desktop/konverto_data_package/polygons/polygons.geojson')

In [3]:
gdf

,type,geometry
0,winery,MULTIPOLYGON EMPTY
1,winery,"MULTIPOLYGON (((675007.413 5143326.011, 675067..."
2,winery,"MULTIPOLYGON (((675066.550 5143267.678, 675149..."
3,winery,"MULTIPOLYGON (((674760.935 5143198.037, 674834..."
4,winery,"MULTIPOLYGON (((673672.158 5143505.380, 673678..."
...,...,...
96,apple orchard,"MULTIPOLYGON (((673977.481 5136172.898, 674005..."
97,winery,"MULTIPOLYGON (((673322.831 5136427.445, 673328..."
98,winery,"MULTIPOLYGON (((672685.775 5139609.934, 672709..."
99,apple orchard,"MULTIPOLYGON (((678006.627 5145427.665, 677996..."


In [4]:
gdf1 = gdf.loc[11]
gdf1

type                                                   winery
geometry    MULTIPOLYGON (((673161.0209203674 5138097.8153...
Name: 11, dtype: object

In [33]:
print(gdf.loc[8].geometry)


MULTIPOLYGON (((673470.9524184616 5139569.188658351, 673539.6077218383 5139714.539462282, 673668.12560034 5139682.868027472, 673651.4405178521 5139607.648878803, 673660.532443122 5139604.252607453, 673654.8981749808 5139584.999839183, 673616.7767223015 5139522.16748573, 673470.9524184616 5139569.188658351)))


In [6]:
raster = rasterio.open('../../../Desktop/konverto_data_package/climate/tmean/tmean_2020_01_21.tif')
raster

<open DatasetReader name='../../../Desktop/konverto_data_package/climate/tmean/tmean_2020_01_21.tif' mode='r'>

In [7]:
out, inside  = mask(raster, gdf1[['geometry']], crop=True, invert=False, all_touched=True, nodata=np.nan)
out

array([[[       nan, 0.9553406 ],
        [0.88942724, 0.7424053 ],
        [0.8325057 , 0.34043524]]], dtype=float32)

In [27]:
def get_dataset_file_paths():

    directory = "../../../Desktop/konverto_data_package"
    
    datasets = {}
    for root, dirs, files in os.walk(directory):
        # print(f"Current directory: {root}")
        if "visual" in root:
            continue

        tif_files = [f for f in files if f.endswith(".tif") and not f.startswith(".")]
        tif_paths = [os.path.join(root, f) for f in tif_files]


        if len(tif_files) > 0:

            if "climate" in tif_paths[0]:
                dates = ["-".join(f.replace(".tif", "").split("_")[1:]) for f in tif_files]
            elif "satellite" in tif_paths[0]:
                datestrings = [f.replace(".tif", "").split("_")[0] for f in tif_files]
                dates = [d[:4] + "-" + d[4:6] + "-" + d[6:] for d in datestrings]
            else:
                dates = None
            

            if dates:
                dataset_name = os.path.basename(root)
                
                for date, fp in zip(dates, tif_paths):
                    if date in datasets:
                        datasets[date].update({dataset_name: fp})
                    else:
                        datasets[date] = {dataset_name: fp}
            
            if "terrain" in tif_paths[0]:
                dataset_names = [n.replace(".tif", "") for n in tif_files]
                for date in datasets.keys():
                    for dn, fp in zip(dataset_names, tif_paths):
                        datasets[date].update({dn: fp})

                    


    return datasets

In [28]:
datasets = get_dataset_file_paths()
datasets

{'2021-05-03': {'reci': '../../../Desktop/konverto_data_package/satellite/indices/reci/20210503_reci.tif',
  'ndwi': '../../../Desktop/konverto_data_package/satellite/indices/ndwi/20210503_ndwi.tif',
  'ndvi': '../../../Desktop/konverto_data_package/satellite/indices/ndvi/20210503_ndvi.tif',
  'gdd': '../../../Desktop/konverto_data_package/climate/gdd/gdd_2021_05_03.tif',
  'tmax': '../../../Desktop/konverto_data_package/climate/tmax/tmax_2021_05_03.tif',
  'tmin': '../../../Desktop/konverto_data_package/climate/tmin/tmin_2021_05_03.tif',
  'tmean': '../../../Desktop/konverto_data_package/climate/tmean/tmean_2021_05_03.tif',
  'pmm': '../../../Desktop/konverto_data_package/climate/pmm/pmm_2021_05_03.tif'},
 '2021-08-08': {'reci': '../../../Desktop/konverto_data_package/satellite/indices/reci/20210808_reci.tif',
  'ndwi': '../../../Desktop/konverto_data_package/satellite/indices/ndwi/20210808_ndwi.tif',
  'ndvi': '../../../Desktop/konverto_data_package/satellite/indices/ndvi/20210808_nd

In [29]:
datasets["2020-03-22"]

{'reci': '../../../Desktop/konverto_data_package/satellite/indices/reci/20200322_reci.tif',
 'ndwi': '../../../Desktop/konverto_data_package/satellite/indices/ndwi/20200322_ndwi.tif',
 'ndvi': '../../../Desktop/konverto_data_package/satellite/indices/ndvi/20200322_ndvi.tif',
 'gdd': '../../../Desktop/konverto_data_package/climate/gdd/gdd_2020_03_22.tif',
 'tmax': '../../../Desktop/konverto_data_package/climate/tmax/tmax_2020_03_22.tif',
 'tmin': '../../../Desktop/konverto_data_package/climate/tmin/tmin_2020_03_22.tif',
 'tmean': '../../../Desktop/konverto_data_package/climate/tmean/tmean_2020_03_22.tif',
 'pmm': '../../../Desktop/konverto_data_package/climate/pmm/pmm_2020_03_22.tif'}

In [30]:
def extract_raster_data_from_polygon(polygon, paths_data, stat = "mean"):

    data = {}
    mean = {}
    std = {}
    for dataset_name, fpath in paths_data.items():
        raster = rasterio.open(fpath)
        if stat == "mean":
            data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
        elif stat == "std":
            data[dataset_name] = np.nanstd(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
        elif stat == "max":
            data[dataset_name] = np.nanmax(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
        elif stat == "min":
            data[dataset_name] = np.nanmin(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
        elif stat == "mean_std":
            mean[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
            std[dataset_name] = np.nanstd(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
            data = (mean, std)



    return data


def get_time_series_data_from_polygon(polygon, stat="mean"):
    
    paths_dataset = get_dataset_file_paths()
    dti = pd.date_range("2020-03-22", end="2022-12-31", freq="1D")
    rows = []
    for d in dti:
        try:
            date_string = d.strftime("%Y-%m-%d")
            row = extract_raster_data_from_polygon(polygon, paths_dataset[date_string], stat)
            rows.append(row)
        except:
            print(d)
            rows.append({
                    "gdd": np.nan,
                    "pmm": np.nan,
                    "tmax": np.nan,
                    "tmean": np.nan,
                    "tmin": np.nan,
                    "elevation": np.nan,
                    "exposure": np.nan,
                    "slope": np.nan,
                    "ndvi": np.nan,
                    "ndwi": np.nan,
                    "reci": np.nan,
                })
    ts = pd.DataFrame(index=dti,data=rows)

    return ts
    


In [31]:
ts = get_time_series_data_from_polygon(gdf.loc[8][['geometry']])


/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])


2020-11-05 00:00:00


/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_1

2021-07-03 00:00:00


/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_1

2022-06-16 00:00:00
2022-06-28 00:00:00


/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_14275/658299066.py:9: RuntimeWarning: Mean of empty slice
  data[dataset_name] = np.nanmean(mask(raster, polygon, crop=True, invert=False, all_touched=True, nodata=np.nan)[0])
/var/folders/vy/1_ps96bj3vn81l5_z37hplc80000gn/T/ipykernel_1

In [32]:
ts.head()

,reci,ndwi,ndvi,gdd,tmax,tmin,tmean,pmm
2020-03-22,0.502375,-0.062845,0.200307,3.051428,18.794580,7.310073,12.797966,0.524082
2020-04-03,NaN,NaN,NaN,0.418571,19.729502,1.106295,10.133245,0.000000
2020-04-15,NaN,NaN,NaN,2.890000,21.229456,4.547482,12.031464,0.000000
2020-04-27,NaN,NaN,NaN,7.941429,22.510900,13.374471,17.254629,0.000158
2020-05-09,NaN,NaN,NaN,9.821428,27.441988,12.198226,19.986624,0.000000


In [35]:
ts

,reci,ndwi,ndvi,gdd,tmax,tmin,tmean,pmm
2020-03-22,0.502375,-0.062845,0.200307,3.051428,18.794580,7.310073,12.797966,0.524082
2020-04-03,NaN,NaN,NaN,0.418571,19.729502,1.106295,10.133245,0.000000
2020-04-15,NaN,NaN,NaN,2.890000,21.229456,4.547482,12.031464,0.000000
2020-04-27,NaN,NaN,NaN,7.941429,22.510900,13.374471,17.254629,0.000158
2020-05-09,NaN,NaN,NaN,9.821428,27.441988,12.198226,19.986624,0.000000
...,...,...,...,...,...,...,...,...
2022-11-07,0.763480,-0.125700,0.275229,0.000000,14.006971,2.866947,7.368102,0.000000
2022-11-19,0.865843,-0.153502,0.300578,0.000000,12.820563,1.671435,6.521374,0.000000
2022-12-01,0.771424,-0.150571,0.276883,0.000000,8.514242,-0.058092,3.095683,0.000000
2022-12-13,NaN,NaN,NaN,0.000000,0.008598,-6.274934,-2.192403,0.000000


In [42]:
data_ = gpd.read_file('../complete.geojson')
print(data_.name.unique())
for index, _ in data_.iterrows():
    df = get_time_series_data_from_polygon(data_["geometry"][index])
    print(df)
    break



['P_431-6_681' 'P_3596_621' 'P_317_823' ... 'P_3748-1_621' 'P_4084_621'
 'P_3955-2_621']
2020-03-22 00:00:00


UnboundLocalError: cannot access local variable 'row' where it is not associated with a value